In [1]:
from bayescache.data import P3B3
from bayescache.api.data import train_valid_split
from bayescache.api.data.source import TrainingData

from torch.utils.data import DataLoader

In [2]:
train = P3B3('/Users/yngtodd/data', partition='train', download=True)

In [4]:
trainloader = DataLoader(train, batch_size=1)

In [6]:
for idx, (data, labels) in enumerate(trainloader):
    print(labels)
    if idx == 2:
        break

{'subsite': tensor([5]), 'laterality': tensor([0]), 'histology': tensor([2]), 'grade': tensor([2])}
{'subsite': tensor([4]), 'laterality': tensor([0]), 'histology': tensor([2]), 'grade': tensor([1])}
{'subsite': tensor([5]), 'laterality': tensor([0]), 'histology': tensor([1]), 'grade': tensor([0])}


In [7]:
train, valid = train_valid_split(train, valpercent=.25)

In [12]:
len(train)

6000

In [13]:
len(valid)

2000

In [8]:
traindata = TrainingData(train, .25, 2, 1)

In [9]:
val_loader = traindata.val_loader()

In [11]:
for idx, (data, label) in enumerate(val_loader):
    print(f'idx: {idx}, label: {label}')
    if idx == 2:
        break

idx: 0, label: {'subsite': tensor([5]), 'laterality': tensor([0]), 'histology': tensor([1]), 'grade': tensor([2])}
idx: 1, label: {'subsite': tensor([4]), 'laterality': tensor([0]), 'histology': tensor([2]), 'grade': tensor([1])}
idx: 2, label: {'subsite': tensor([0]), 'laterality': tensor([0]), 'histology': tensor([2]), 'grade': tensor([2])}


In [14]:
valid = traindata.val_dataset()

In [15]:
len(valid)

1500

In [16]:
train = traindata.train_dataset()

In [17]:
len(train)

4500

In [18]:
source = traindata.train_loader()

In [20]:
for idx, (data, labels) in enumerate(source):
    print(f'idx: {idx}, label: {label}')
    if idx == 2:
        break

idx: 0, label: {'subsite': tensor([0]), 'laterality': tensor([0]), 'histology': tensor([2]), 'grade': tensor([2])}
idx: 1, label: {'subsite': tensor([0]), 'laterality': tensor([0]), 'histology': tensor([2]), 'grade': tensor([2])}
idx: 2, label: {'subsite': tensor([0]), 'laterality': tensor([0]), 'histology': tensor([2]), 'grade': tensor([2])}
